In [1]:
import os, json
import pandas as pd
import numpy as np
import cv2
import math

Path to folder with json files

In [2]:
path_to_ballet = 'C:\\Users\\user\\Desktop\\project\\output\\ballet'
path_to_contemporary='C:\\Users\\user\\Desktop\\project\\output\\contemporary'
path_to_contemporary2='C:\\Users\\user\\Desktop\\project\\output\\contemporary21'
path_to_contemporary3='C:\\Users\\user\\Desktop\\project\\output\\contemporary22'

In [3]:
path_to_ballet_2 ='C:\\Users\\user\\Desktop\\project\\1453169-20200914T174236Z-001\\TB_S'

In [4]:
path_to_contemporary_2='C:\\Users\\user\\Desktop\\project\\1453169-20200914T174236Z-001\\TB_S_FB'

It creates a dataframe with all the angles for a frame. So it is the angles for the whole video from -1 to 1 values.

In [5]:
def create_df(path):
    newDF = pd.DataFrame(index=range(29))
    i=0
    
    
    for data in os.listdir(path):
        data=open(os.path.join(path,data),'r')
        data=json.load(data)
    
        def read_keypoints(data):
            people_column=data['people']
            for d in people_column:
                keypoints=(d["pose_keypoints_2d"])
            return keypoints  
    
    
        def remove_confidence_interval(data):
        
            j=2
            keypoints=read_keypoints(data)
            keypoints_woconfidence=keypoints.copy()
            while j<=len(keypoints_woconfidence):
                keypoints_woconfidence.pop(j)
                j+=2
            return keypoints_woconfidence
        
    
            
        def create_y_coordicate(data):
            df=remove_confidence_interval(data)
            df=np.array(df)
            mask = np.ones(df.size, dtype=bool)
            mask[1::2] = 0
            points = df[mask]
            y=points.tolist()
            return y
        
        def create_x_coordicate(data):
            df=remove_confidence_interval(data)
            df=np.array(df)
            mask = np.zeros(df.size, dtype=bool)
            mask[1::2] = 1
            points = df[mask]
            x=points.tolist()
            return x
        
        
        def bodyparts(data):        
            
            # //     {0,  "Nose"}, 
            # //     {1,  "Neck"},
            # //     {2,  "RShoulder"},
            # //     {3,  "RElbow"},
            # //     {4,  "RWrist"},
            # //     {5,  "LShoulder"},
            # //     {6,  "LElbow"},
            # //     {7,  "LWrist"},
            # //     {8,  "MidHip"},
            # //     {9,  "RHip"},
            # //     {10, "RKnee"},
            # //     {11, "RAnkle"},
            # //     {12, "LHip"},
            # //     {13, "LKnee"},
            # //     {14, "LAnkle"},
            # //     {15, "REye"},
            # //     {16, "LEye"},
            # //     {17, "REar"},
            # //     {18, "LEar"},
            # //     {19, "LBigToe"},
            # //     {20, "LSmallToe"},
            # //     {21, "LHeel"},
            # //     {22, "RBigToe"},
            # //     {23, "RSmallToe"},
            # //     {24, "RHeel"},
            # //     {25, "Background"}    
            
            x=create_x_coordicate(data)
            y=create_y_coordicate(data)
            
        
            nose_to_neck=(x[0]-x[1]),(y[0]-y[1])
            
            right_eye_to_nose=(x[0]-x[15]),(y[0]-y[15])
            
            right_ear_to_eye=(x[15]-x[17]),(y[15]-y[17])
            left_eye_to_nose=(x[0]-x[16]),(y[0]-y[16])
            left_ear_to_eye=(x[16]-x[18]),(y[16]-y[18])
            
            torso= (x[1]-x[8]),(y[1]-y[8])
            
            
            right_shoulder=(x[1]-x[2]),(y[1]-y[2])
            left_shoulder=(x[1]-x[5]),(y[1]-y[5])
            
            right_upper_arm=(x[2]-x[3]),(y[2]-y[3])
            right_lower_arm=(x[3]-x[4]),(y[3]-y[4])
            left_upper_arm=(x[5]-x[6]),(y[5]-y[6])
            left_lower_arm=(x[6]-x[7]),(y[6]-y[7])
            
            right_hip=(x[8]-x[9]),(y[8]-y[9])
            right_upper_leg=(x[9]-x[10]),(y[9]-y[10])
            right_lower_leg=(x[10]-x[11]),(y[10]-y[11])
            left_hip=(x[8]-x[12]),(y[8]-y[12])
            left_upper_leg=(x[12]-x[13]),(y[12]-y[13])
            left_lower_leg=(x[13]-x[14]),(y[13]-y[14])
            
            left_foot=(x[14]-x[19]),(y[14]-y[19])
            left_toes=(x[19]-x[20]),(y[19]-y[20])
            left_ankle_to_heel=(x[14]-x[21]),(y[14]-y[21]) 
            right_foot=(x[11]-x[22]),(y[11]-y[22])
            right_toes=(x[22]-x[23]),(y[22]-y[23])
            right_ankle_to_heel=(x[11]-x[24]),(y[11]-y[24])
            
            body_vector=np.array([nose_to_neck, left_eye_to_nose ,left_ear_to_eye, right_eye_to_nose,right_ear_to_eye, torso,left_shoulder, right_shoulder,right_upper_arm ,right_lower_arm ,left_upper_arm ,left_lower_arm,left_hip ,left_upper_leg ,left_lower_leg ,right_hip ,right_upper_leg,right_lower_leg,right_foot ,right_toes ,right_ankle_to_heel ,left_foot ,left_toes ,left_ankle_to_heel])
            return body_vector
        
        

        #ANGLE SIMILARITY
        
        #defines the degree of an angle    
        import math
        def degree(x):
            pi=math.pi
            degree=(x*180)/pi
            return degree
        
        
        #produces RunitmeWarning for division with zero
        np.seterr(divide='ignore', invalid='ignore')
        
        #for every neighbor vectors compute angles
        
        def compute_angle(vector1,vector2):   
            unit_vector_1 = vector1/ np.linalg.norm(vector1)
            unit_vector_2 = vector2/ np.linalg.norm(vector2)
            dot_product = np.dot(unit_vector_1, unit_vector_2)
            angle = np.arccos(dot_product)
            return round(math.cos(angle),3)
        
        #create a vector with all angles
        def compute_angle_vector(data):
            x=create_x_coordicate(data)
            y=create_y_coordicate(data)
            
            nose_to_neck=(x[0]-x[1]),(y[0]-y[1])
            
            right_shoulder=(x[1]-x[2]),(y[1]-y[2])
            left_shoulder=(x[1]-x[5]),(y[1]-y[5])
              
            angle_nose_to_neck_to_left_shoulder=compute_angle(nose_to_neck,left_shoulder)
            angle_nose_to_neck_to_right_shoulder=compute_angle(nose_to_neck,right_shoulder)
            angle_left_shoulder_to_right_shoulder=compute_angle(left_shoulder,right_shoulder)
            
            left_upper_arm=(x[5]-x[6]),(y[5]-y[6])
            angle_left_shoulder_to_left_upper_arm=compute_angle(left_upper_arm,left_shoulder)
            
            left_lower_arm=(x[6]-x[7]),(y[6]-y[7])
            angle_left_lower_arm_to_left_upper_arm=compute_angle(left_upper_arm,left_lower_arm)
            
            right_upper_arm=(x[2]-x[3]),(y[2]-y[3])
            angle_right_upper_arm_to_right_shoulder=compute_angle(right_upper_arm,right_shoulder)
            
            right_lower_arm=(x[3]-x[4]),(y[3]-y[4])
            angle_right_upper_arm_to_right_lower_arm=compute_angle(right_upper_arm,right_lower_arm)
            
            left_eye_to_nose=(x[0]-x[16]),(y[0]-y[16])
            left_ear_to_eye=(x[16]-x[18]),(y[16]-y[18])
            
            angle_left_eye_to_nose_to_left_ear_to_eye=compute_angle(left_eye_to_nose,left_ear_to_eye)
            
            angle_left_eye_to_nose_to_neck=compute_angle(left_eye_to_nose,nose_to_neck)
        
            right_eye_to_nose=(x[0]-x[15]),(y[0]-y[15])
            right_ear_to_eye=(x[15]-x[17]),(y[15]-y[17])
        
            angle_nose_to_neck_to_right_eye_to_nose=compute_angle(nose_to_neck,right_eye_to_nose)  
            angle_left_eye_to_nose_to_rigt_ear_to_eye=compute_angle(left_eye_to_nose,right_ear_to_eye)
            
            angle_right_eye_to_nose_to_right_ear_to_eye=compute_angle(right_eye_to_nose,right_ear_to_eye)
        
            right_hip=(x[8]-x[9]),(y[8]-y[9])
            right_upper_leg=(x[9]-x[10]),(y[9]-y[10])
            
            angle_right_hip_to_right_upper_leg=compute_angle(right_hip,right_upper_leg)
            
            right_lower_leg=(x[10]-x[11]),(y[10]-y[11])
                
            angle_right_upper_leg_to_right_lower_leg=compute_angle(right_upper_leg,right_lower_leg)    
            
            left_hip=(x[8]-x[12]),(y[8]-y[12])
            left_upper_leg=(x[12]-x[13]),(y[12]-y[13])
            
            angle_left_hip_to_left_upper_leg=compute_angle(left_hip,left_upper_leg)
            
            left_lower_leg=(x[13]-x[14]),(y[13]-y[14])
            
            angle_left_upper_leg_to_left_lower_leg=compute_angle(left_upper_leg,left_lower_leg)
            
            left_ankle_to_heel=(x[14]-x[21]),(y[14]-y[21]) 
            
            angle_left_lower_leg_left_ankle_to_heel=compute_angle(left_lower_leg,left_ankle_to_heel)
            
            right_ankle_to_heel=(x[11]-x[24]),(y[11]-y[24])
            
            angle_right_lower_leg_to_right_ankle_to_heel=compute_angle(right_lower_leg,right_ankle_to_heel)
            
            right_toes=(x[22]-x[23]),(y[22]-y[23])
            right_foot=(x[11]-x[22]),(y[11]-y[22])
            
            angle_right_foot_to_right_toes=compute_angle(right_foot,right_toes)
            angle_right_foot_to_right_lower_leg=compute_angle(right_foot,right_lower_leg)
            angle_right_foot_to_right_ankle_to_heel=compute_angle(right_foot,right_ankle_to_heel)
            
            left_foot=(x[14]-x[19]),(y[14]-y[19])
            left_toes=(x[19]-x[20]),(y[19]-y[20])
            
            angle_left_foot_to_left_lower_leg=compute_angle(left_foot, left_lower_leg)
            angle_left_foot_to_left_ankle_to_heel=compute_angle(left_foot, left_ankle_to_heel)
            angle_left_foot_to_left_toes=compute_angle(left_foot,left_toes)
            
            torso= (x[1]-x[8]),(y[1]-y[8])
            
            angle_torso_to_right_shoulder=compute_angle(torso,right_shoulder)
            angle_torso_to_left_shoulder=compute_angle(torso,left_shoulder)
            angle_torso_to_nose_to_neck=compute_angle(torso,nose_to_neck)
            
            angle_torso_to_right_hip=compute_angle(torso,right_hip)
            angle_torso_to_left_hip=compute_angle(torso,left_hip)
            
            
            body_vector=np.array([angle_nose_to_neck_to_left_shoulder,angle_nose_to_neck_to_right_shoulder,angle_left_shoulder_to_right_shoulder, angle_left_shoulder_to_left_upper_arm,angle_left_lower_arm_to_left_upper_arm,angle_right_upper_arm_to_right_shoulder,angle_right_upper_arm_to_right_lower_arm, angle_left_eye_to_nose_to_left_ear_to_eye, angle_left_eye_to_nose_to_neck,angle_nose_to_neck_to_right_eye_to_nose, angle_left_eye_to_nose_to_rigt_ear_to_eye, angle_right_eye_to_nose_to_right_ear_to_eye,angle_right_hip_to_right_upper_leg,angle_right_upper_leg_to_right_lower_leg, angle_left_hip_to_left_upper_leg,angle_left_upper_leg_to_left_lower_leg,angle_left_lower_leg_left_ankle_to_heel, angle_right_lower_leg_to_right_ankle_to_heel, angle_right_foot_to_right_toes, angle_right_foot_to_right_lower_leg, angle_right_foot_to_right_ankle_to_heel, angle_left_foot_to_left_lower_leg,  angle_left_foot_to_left_ankle_to_heel,  angle_left_foot_to_left_toes ,angle_torso_to_right_shoulder ,angle_torso_to_left_shoulder , angle_torso_to_nose_to_neck, angle_torso_to_right_hip ,angle_torso_to_left_hip  ])
            return body_vector
            
        bodyvector1=compute_angle_vector(data)
        new_bodyvector=pd.DataFrame(bodyvector1)
        
        newDF['Frame: ',i]=new_bodyvector
        #newDF[i]=new_bodyvector
        i+=1
        #print(bodyvector1)
        
    return newDF

In [6]:
#ballet_df=create_df(path_to_ballet)
#contemporary_df=create_df(path_to_contemporary)     
#contemporary_df2=create_df(path_to_contemporary2)  
#contemporary_df3=create_df(path_to_contemporary3) 

In [7]:
ballet_df=create_df(path_to_ballet_2)
contemporary_df=create_df(path_to_contemporary_2)  

In [8]:
ballet_df[:5]

,"(Frame: , 0)","(Frame: , 1)","(Frame: , 2)","(Frame: , 3)","(Frame: , 4)","(Frame: , 5)","(Frame: , 6)","(Frame: , 7)","(Frame: , 8)","(Frame: , 9)",...,"(Frame: , 89)","(Frame: , 90)","(Frame: , 91)","(Frame: , 92)","(Frame: , 93)","(Frame: , 94)","(Frame: , 95)","(Frame: , 96)","(Frame: , 97)","(Frame: , 98)"
0,0.026,0.026,0.051,0.052,0.037,0.113,0.205,0.276,0.256,0.228,...,-0.029,-0.076,-0.029,-0.027,-0.029,-0.057,-0.029,-0.027,0.019,-0.000
1,-0.068,-0.068,-0.093,-0.054,-0.118,-0.155,-0.206,-0.277,-0.301,-0.278,...,0.026,-0.022,0.028,-0.025,-0.022,0.056,-0.020,-0.022,0.026,-0.000
2,-0.999,-0.999,-0.999,-1.000,-0.997,-0.999,-1.000,-1.000,-0.999,-0.999,...,-1.000,-0.995,-1.000,-0.999,-0.999,-1.000,-0.999,-0.999,-0.999,-1.000
3,0.961,0.949,0.935,0.914,0.839,0.726,0.640,0.224,-0.248,-0.441,...,0.668,0.618,0.689,0.707,0.674,0.690,0.672,0.708,0.756,0.725
4,0.978,0.970,0.961,0.976,0.953,0.883,0.865,0.995,-0.085,0.314,...,0.992,0.968,0.978,0.989,0.956,0.961,0.941,0.966,0.962,0.956


In [9]:
contemporary_df[:5]

,"(Frame: , 0)","(Frame: , 1)","(Frame: , 2)","(Frame: , 3)","(Frame: , 4)","(Frame: , 5)","(Frame: , 6)","(Frame: , 7)","(Frame: , 8)","(Frame: , 9)",...,"(Frame: , 91)","(Frame: , 92)","(Frame: , 93)","(Frame: , 94)","(Frame: , 95)","(Frame: , 96)","(Frame: , 97)","(Frame: , 98)","(Frame: , 99)","(Frame: , 100)"
0,-0.923,-0.923,-0.906,-0.906,-0.921,-0.904,-0.919,-0.919,-0.917,-0.916,...,-0.898,-0.900,-0.902,-0.904,-0.925,-0.908,-0.910,-0.928,-0.929,-0.929
1,0.926,0.925,0.925,0.941,0.923,0.939,0.938,0.937,0.919,0.918,...,0.917,0.900,0.901,0.923,0.925,0.928,0.929,0.910,0.912,0.911
2,-1.000,-1.000,-0.999,-0.996,-1.000,-0.996,-0.999,-0.999,-1.000,-1.000,...,-0.999,-1.000,-1.000,-0.999,-1.000,-0.999,-0.999,-0.999,-0.999,-0.999
3,0.925,0.927,0.946,0.948,0.934,0.952,0.941,0.942,0.949,0.949,...,0.720,0.720,0.720,0.707,0.658,0.694,0.692,0.658,0.659,0.659
4,0.985,0.987,0.982,0.979,0.973,0.976,0.977,0.978,0.986,0.986,...,0.968,0.963,0.962,0.949,0.925,0.925,0.943,0.943,0.958,0.943


DTW implementation

In [10]:
def dtw2(x, y):
    # Initialization
    n, m = len(x), len(y)
    dtw_matrix = np.zeros((n+1, m+1))
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
            
    dtw_matrix[0, 0] = 0

   # Main loop
    for i in range(1,n+1):
        for j in range(1,m+1):
            cost = (x[i-1] - y[j-1])**2
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min

    return math.sqrt(dtw_matrix[i, j])

In [11]:
dtw2([1,2], [1,2,5,6])

5.0

DTW for multidimensional series

In [12]:
from tslearn.metrics import dtw, dtw_path

In [13]:
dtw([1,2], [1,2,5,6])

5.0

I transpose them in order to put frames in rows

In [14]:
X=ballet_df.T
Y=contemporary_df.T

In [16]:
#Z=contemporary_df2.T
#P=contemporary_df3.T

In [17]:
X.iloc[:2,:5]

,0,1,2,3,4
"(Frame: , 0)",0.026,-0.068,-0.999,0.961,0.978
"(Frame: , 1)",0.026,-0.068,-0.999,0.949,0.970


In [18]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
"(Frame: , 0)",0.026,-0.068,-0.999,0.961,0.978,0.613,-0.218,0.267,-0.608,-0.576,...,0.564,-0.053,0.339,-0.382,-0.761,-0.018,-0.024,0.999,0.026,-0.024
"(Frame: , 1)",0.026,-0.068,-0.999,0.949,0.970,0.632,-0.177,0.145,-0.684,-0.576,...,0.589,0.103,0.366,-0.366,-0.841,-0.030,-0.012,0.999,0.014,-0.012
"(Frame: , 2)",0.051,-0.093,-0.999,0.935,0.961,0.648,-0.280,0.326,-0.586,-0.660,...,0.590,-0.053,0.367,-0.285,-0.841,-0.042,0.000,0.999,0.000,0.000
"(Frame: , 3)",0.052,-0.054,-1.000,0.914,0.976,0.706,-0.438,0.275,-0.668,-0.563,...,0.590,0.028,0.381,-0.365,-0.824,-0.001,-0.000,0.999,0.001,-0.001
"(Frame: , 4)",0.037,-0.118,-0.997,0.839,0.953,0.702,-0.082,0.360,-0.649,-0.686,...,0.588,0.027,0.424,-0.348,-0.813,-0.053,-0.029,0.998,-0.011,-0.057


Since the NAN values will cause problem in the implementation, I put 0 to all NaN values given that we have values from -1 to 1.

In [19]:
X=X.fillna(0)
Y=Y.fillna(0)

In [21]:
#Z=Z.fillna(0)
#P=P.fillna(0)

In [22]:
dtw(X,Y)

27.110333454238436

In [23]:
dtw(Y,X)

27.110333454238436

In [24]:
X

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
"(Frame: , 0)",0.026,-0.068,-0.999,0.961,0.978,0.613,-0.218,0.267,-0.608,-0.576,...,0.564,-0.053,0.339,-0.382,-0.761,-0.018,-0.024,0.999,0.026,-0.024
"(Frame: , 1)",0.026,-0.068,-0.999,0.949,0.970,0.632,-0.177,0.145,-0.684,-0.576,...,0.589,0.103,0.366,-0.366,-0.841,-0.030,-0.012,0.999,0.014,-0.012
"(Frame: , 2)",0.051,-0.093,-0.999,0.935,0.961,0.648,-0.280,0.326,-0.586,-0.660,...,0.590,-0.053,0.367,-0.285,-0.841,-0.042,0.000,0.999,0.000,0.000
"(Frame: , 3)",0.052,-0.054,-1.000,0.914,0.976,0.706,-0.438,0.275,-0.668,-0.563,...,0.590,0.028,0.381,-0.365,-0.824,-0.001,-0.000,0.999,0.001,-0.001
"(Frame: , 4)",0.037,-0.118,-0.997,0.839,0.953,0.702,-0.082,0.360,-0.649,-0.686,...,0.588,0.027,0.424,-0.348,-0.813,-0.053,-0.029,0.998,-0.011,-0.057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(Frame: , 94)",-0.057,0.056,-1.000,0.690,0.961,0.616,0.934,-0.102,-0.808,-0.582,...,0.279,-0.051,0.923,0.783,-0.506,-0.001,-0.001,0.998,0.001,-0.001
"(Frame: , 95)",-0.029,-0.020,-0.999,0.672,0.941,0.554,0.951,-0.001,-0.727,-0.759,...,0.244,-0.126,0.893,0.487,-0.420,-0.049,-0.000,1.000,0.002,-0.001
"(Frame: , 96)",-0.027,-0.022,-0.999,0.708,0.966,0.530,0.932,0.105,-0.649,-0.687,...,0.383,-0.010,0.663,-0.193,-0.617,-0.050,0.001,1.000,0.001,-0.001
"(Frame: , 97)",0.019,0.026,-0.999,0.756,0.962,0.586,0.938,0.027,-0.645,-0.584,...,0.426,-0.028,0.409,-0.412,-0.805,-0.002,0.047,1.000,0.002,-0.001


In [25]:
X[:10]

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
"(Frame: , 0)",0.026,-0.068,-0.999,0.961,0.978,0.613,-0.218,0.267,-0.608,-0.576,...,0.564,-0.053,0.339,-0.382,-0.761,-0.018,-0.024,0.999,0.026,-0.024
"(Frame: , 1)",0.026,-0.068,-0.999,0.949,0.970,0.632,-0.177,0.145,-0.684,-0.576,...,0.589,0.103,0.366,-0.366,-0.841,-0.030,-0.012,0.999,0.014,-0.012
"(Frame: , 2)",0.051,-0.093,-0.999,0.935,0.961,0.648,-0.280,0.326,-0.586,-0.660,...,0.590,-0.053,0.367,-0.285,-0.841,-0.042,0.000,0.999,0.000,0.000
"(Frame: , 3)",0.052,-0.054,-1.000,0.914,0.976,0.706,-0.438,0.275,-0.668,-0.563,...,0.590,0.028,0.381,-0.365,-0.824,-0.001,-0.000,0.999,0.001,-0.001
"(Frame: , 4)",0.037,-0.118,-0.997,0.839,0.953,0.702,-0.082,0.360,-0.649,-0.686,...,0.588,0.027,0.424,-0.348,-0.813,-0.053,-0.029,0.998,-0.011,-0.057
"(Frame: , 5)",0.113,-0.155,-0.999,0.726,0.883,0.731,0.324,0.406,-0.512,-0.880,...,0.538,0.039,0.485,-0.227,-0.789,0.000,-0.043,0.988,0.001,-0.000
"(Frame: , 6)",0.205,-0.206,-1.000,0.640,0.865,0.765,0.961,0.470,-0.463,-0.909,...,0.471,0.009,0.464,-0.262,-0.808,0.013,-0.014,0.976,0.082,-0.013
"(Frame: , 7)",0.276,-0.277,-1.000,0.224,0.995,0.772,0.995,0.662,-0.334,-1.000,...,0.413,-0.115,0.592,-0.170,-0.741,0.012,-0.013,0.957,0.087,-0.013
"(Frame: , 8)",0.256,-0.301,-0.999,-0.248,-0.085,0.708,0.978,0.636,-0.358,-1.000,...,0.382,-0.127,0.549,-0.060,-0.515,0.015,-0.061,0.949,0.088,-0.014
"(Frame: , 9)",0.228,-0.278,-0.999,-0.441,0.314,0.821,0.975,0.603,-0.375,-0.711,...,0.349,-0.249,0.519,-0.167,-0.716,0.147,-0.198,0.909,0.105,-0.026


In [26]:
dtw_path(X[:11],X[:10])

([(0, 0),
  (1, 1),
  (2, 2),
  (3, 3),
  (4, 4),
  (5, 5),
  (6, 6),
  (7, 7),
  (8, 8),
  (9, 9),
  (10, 9)],
 1.466109477494774)

TO DO: Further search about constraints,theory,..

In [27]:
dtw(X,Y,global_constraint ="itakura")

27.921882207329794

The optimal alligned frames and the dissimilarity number are:

In [30]:
dtw(X,Y)

27.110333454238436

In [31]:
dtw(Y,X)

27.110333454238436

In [ ]:
 dtw_path(X,Y)

The optimal allignment between frames of two videos is:

In [ ]:
dtw_path(X,Y)[0]

In [ ]:
dtw_path(X,Y)[0][0]

In [ ]:
L=dtw_path(X,Y)[0]

In [ ]:
for i,j in L:
    
    print(i,j)

In [ ]:
print("The DTW dissimilarity for ballet_df and contemporary_df is :",dtw(X,Y))

In [ ]:
print("The DTW dissimilarity for ballet_df and another contemporary_df is :",dtw(X,Z))

In [ ]:
print("The DTW dissimilarity for two different contemporary_df is :",dtw(Y,Z))

In [ ]:
print("The DTW dissimilarity for two consequent contemporary_df is :",dtw(P,Z))

In [ ]:
df = pd.DataFrame(dtw_path(X,Y)[-2])

In [ ]:
df.columns =['video1', 'video2'] 

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df["video1"], df["video2"])
plt.show() 

In [ ]:
import seaborn as sns
sns.scatterplot(x=df["video1"], y=df["video2"], data=df)

In [ ]:
#plot signals
plt.plot(df["video1"].values, label = 'Video 1')
plt.plot(df["video2"].values, label = 'Video 2')
for i, j in dtw_path(X,Y)[0]:
    plt.plot((i, j), (df["video1"].values[i], df["video2"].values[j]))
plt.margins(x=0, y=0) 
plt.show()

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="video1", y="video2")
fig.show()


In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=df["video1"].values, y=df["video2"].values,
                    mode='lines+markers',
                    name='lines+markers'))

fig.show()

Another implementation of MD-DTW to prove its correctness.

In [32]:
def MDTWDistance(s1, s2, window=10, num_columns=1):
    DTW={}

    w = max(window, abs(len(s1)-len(s2)))

    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= mdist(s1.loc[i], s2.loc[j], num_columns)
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return np.sqrt(DTW[len(s1)-1, len(s2)-1])

def mdist(a, b, num_col):
    dist = 0
    for col in range(num_col):
        #print "Finding Distance of", a[col], b[col]
        dist = dist + (a[col]-b[col])**2
    return dist

In [33]:
MDTWDistance(X,Y,num_columns=29)

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [0] of <class 'int'>

In [34]:
MDTWDistance(X,Z,num_columns=29)

NameError: name 'Z' is not defined

Next steps: Compute similarity between 2 alligned frames and plot them.
Think of what to do with null values?
PCA for angles?